In [1]:
import sys
import pyzed.sl as sl
import cv2

import math
import numpy as np
import random
from matplotlib import pyplot as plt
import time

import get_clear_hand_mask
import find_fingertip
import tracking
import touched_detection
import socket_sender


In [2]:
def candy_edge_with_skin_detect(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb) # 把圖像轉換到YUV色域
    (y, cr, cb) = cv2.split(ycrcb) # 圖像分割, 分別獲取y, cr, br通道圖像
    # 高斯濾波, cr 是待濾波的源圖像數據, (5,5)是值窗口大小, 0 是指根據窗口大小來計算高斯函數標準差
    kernel_size = 5
    cr1 = cv2.GaussianBlur(cr, (kernel_size, kernel_size), 0) # 對cr通道分量進行高斯濾波
    # 根據OTSU算法求圖像閾值, 對圖像進行二值化_, 
    _ ,skin1 = cv2.threshold(cr1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    
    cv2.imshow("Skin Cr+OSTU", skin1 )
    return skin1

# Transform Data

In [3]:
def send_fingertips_pos(fingertips, threed_points):
    data = ""
    temp = ['1 ']
    for i,tip in enumerate(fingertips):
        (u, v) = (int(tip[0]), int(tip[1]))
        temp.append("%d %d "%(u, v)) # speedup the string append
    data = data.join(temp)
    print('send fingertips pos:', data)
    socket_sender.send(data)
    
def send_touch_events(touch_flag):
    data = ""
    temp = ['2 ']
    for i,touch in enumerate(touch_flag):
        temp.append("%d "%(touch)) # speedup the string append
    data = data.join(temp)
    print('send touch flag:', data)
    socket_sender.send(data)

In [4]:
def main():

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.RESOLUTION_HD720
    init.depth_mode = sl.DEPTH_MODE.DEPTH_MODE_PERFORMANCE
    init.depth_minimum_distance = 150 # Set the minimum depth perception distance to 15cm
    init.coordinate_units = sl.UNIT.UNIT_MILLIMETER#sl.UNIT.UNIT_METER  #sl.UNIT.UNIT_MILLIMETER
#     print(init.depth_mode)#PERFORMANCE

    Streaming = True
    if(Streaming) : 
        #read from streaming
        sys.argv[1] = '127.0.0.1'
        if (len(sys.argv) > 1) :
            ip = sys.argv[1]
            init.set_from_stream(ip,30000)
        else :
            print('Usage : python3 streaming_receiver.py ip')
            exit(1)
    else:   
        #read from SVO file
        filepath = 'record_1028_6.svo'#record_1028_6.svo
        print("Reading SVO file: {0}".format(filepath))
        init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)
    
    

    cam = sl.Camera()
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    runtime.sensing_mode = sl.SENSING_MODE.SENSING_MODE_FILL 
    
    image = sl.Mat() #image
    depth = sl.Mat() #depth map
    depth_for_display = sl.Mat() #depth map,scale its values to [0, 255]
    point_cloud = sl.Mat() #colored point cloud.
    confidence_map = sl.Mat() # confidence_map  [0 ,100], 0 :the best confidence value; 100 :the most unconfident one.

    key = ''
    First = True
    print("  Quit : CTRL+C\n")
    while key != 113:
        err = cam.grab(runtime)
        if (err == sl.ERROR_CODE.SUCCESS) :
            # Retrieve left image
            cam.retrieve_image(image, sl.VIEW.VIEW_LEFT)
            cam.retrieve_image(depth_for_display, sl.VIEW.VIEW_DEPTH)
            
            # Load Image data into a numpy array
            RGB_image = image.get_data()
            gray_image = cv2.cvtColor(RGB_image, cv2.COLOR_BGR2GRAY)
            
            
            # Retrieve depth map. Depth is aligned on the left image
            cam.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
            # Load depth data into a numpy array
            depth_data = depth.get_data()
            
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
            # Load point cloud into a numpy array
            points3D_color = point_cloud.get_data()
            points3D = points3D_color[:,:,:3] # only need x,y,z 
#             points3D = np.where(points3D == np.nan, 1111, points3D)# replace nan to 1111111
            
            
            #get confidence index : index which value is not NAN 
            Confidence_index = np.argwhere(~np.isnan(depth_data[200:520,300:980]))#center region of image
#             Confidence_index = np.argwhere(~np.isnan(depth_data[:520,300:980]))
            
#             # Retrieve confidence_map.
#             cam.retrieve_measure(confidence_map, sl.MEASURE.MEASURE_CONFIDENCE)
#             # Load confidence_map into a numpy array
#             confidence_img = confidence_map.get_data()
#             confidence_index = np.argwhere(confidence_img <= 60)

            print("Start computing...")
            print(depth_data.shape)
            #####
            if First == True:
#                 #RANSAM to get surface plane
#                 surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 30, inliner_threshold = 10)
                First = False
#                 print("firstly RANSAM")
# #                 socket_sender.send("0 firstly RANSAM")
#                 cv2.imshow("plane_mask", plane_mask.astype(np.uint8)*255)
                
            else:
            
                s_time = time.time()
                #Ransam every times ???
#                 Confidence_index = np.argwhere(plane_mask == True)
#                 surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 10, inliner_threshold = 10)

                #
#                 depthImg = get_clear_hand_mask.get_depth_map(points3D, surface_plane)
#                 print('depth: %.4f'%(time.time() - s_time))
        
#                 #Canny edge map(infrared image) + threshold based edge map(depth image)
#                 t_time = time.time()
#                 Cannyedges, Threshold_based_edge, Edge_map, grayImage = get_clear_hand_mask.get_edge_map(gray_image, depthImg)
#                 print('edges: %.4f'%(time.time() - t_time))
                
#                 Test skin_detect than Canny edge
                t_time = time.time()
                skinImg = candy_edge_with_skin_detect(RGB_image)
                print('skinImg: %.4f'%(time.time() - t_time))

# #                 #Get hight region by Hight and record its position
#                 t_time = time.time()
#                 High_region_Image, high_region_list = get_clear_hand_mask.get_high_region(depthImg, plane_mask)
#                 print('High_region: %.4f'%(time.time() - t_time))
        
# #                 #Get Hand mask by Flood fill from high region position with Edge map and check wheather Flood_fill_with_edge_reasonable
#                 t_time = time.time()
#                 Hand_mask_Image = get_clear_hand_mask.get_reasonable_Hand_mask(Edge_map, high_region_list, High_region_Image, points3D)
#                 print('Hand_mask: %.4f'%(time.time() - t_time))
        
#                 #find fingertip
                t_time = time.time()
                ##skinImg
                Hand_mask_Image = skinImg
                ##skinImg without depth
                depthImg = depth_data
                
                cnt, contours_image, hand_center, fingertips = find_fingertip.find_fingertip(Hand_mask_Image)#Hand_mask_Image skinImg
                print('fingertips: %.4f'%(time.time() - t_time))
        
#                 # tracking user hand and fingertips
                t_time = time.time()
                hands_hand, fingertips_pos, tracking_image = tracking.hand_tracking(hand_center, fingertips, points3D, contours_image)
                send_fingertips_pos(fingertips_pos, points3D)
                print('tracking: %.4f'%(time.time() - t_time))
            
#                 # detecte fingers whenther touching the surface
                t_time = time.time()
                touched_flag, touching_detection_image = touched_detection.touching_detection(fingertips_pos, Hand_mask_Image, depthImg, contours_image)
                send_touch_events(touched_flag)
                print('touched_detection: %.4f'%(time.time() - t_time))
                
                print('computing time:', (time.time()-s_time))
        
            #####
                #Check result
#                 cv2.imshow("Depth", depthImg.astype(np.uint8))
# #                 cv2.imshow("plane_mask", plane_mask.astype(np.uint8)*255)
# #                 cv2.imshow("Cannyedges", Cannyedges)
#                 cv2.imshow("Threshold_based_edge", Threshold_based_edge.astype(np.uint8)*255)
#                 cv2.imshow("Edge_map", Edge_map.astype(np.uint8)*255)
# #                 cv2.imshow("High_region_Image", High_region_Image.astype(np.uint8)*255)
# #                 cv2.imshow("Hand_mask_Image", Hand_mask_Image.astype(np.uint8)*255)
                cv2.imshow("tracking_image", tracking_image)
                cv2.imshow("touching_detection_image", touching_detection_image)
                
#                 cv2.imshow("ZED_gray", grayImage)
    
            cv2.imshow("ZED", RGB_image)
#             cv2.imshow("Depth img", depth_for_display.get_data())

            key = cv2.waitKey(1)
        else :
            key = cv2.waitKey(1)
        
        if key == 113:
            cv2.destroyAllWindows()
            break
        
    cam.close()
    

In [5]:
if __name__ == "__main__":
    main()

  Quit : CTRL+C

Start computing...
(376, 672)
Start computing...
(376, 672)
skinImg: 0.0080
fingertips: 0.0140
send fingertips pos: 1 205 86 213 143 250 190 370 204 420 175 431 108 411 75 366 36 353 26 -1 -1 
create socket successful
tracking: 0.0020
265.7699 262.76303 3.0068665
251.60706 250.81725 0.7898102
208.92815 204.77315 4.154999
203.841 203.23027 0.61073303
201.72241 198.60594 3.1164703
touched pos: (250, 190)
touched pos: (370, 204)
touched pos: (431, 108)
touched pos: (411, 75)
touched pos: (366, 36)
send touch flag: 2 0 0 1 1 0 1 1 1 0 0 
touched_detection: 0.0060
computing time: 0.029920339584350586
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0090
send fingertips pos: 1 231 39 210 93 244 188 356 200 422 136 342 18 391 53 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
419.02008 413.52518 5.4949036
313.76068 307.48676 6.273926
275.9282 269.76367 6.1645203
253.31334 252.2829 1.0304413
572.3469 569.1071 3.2398071
218.03857 215.8534 2.1851807
touched pos: (231, 39)
touched

fingertips: 0.0050
send fingertips pos: 1 325 7 386 64 482 86 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
405.64236 402.2952 3.347168
358.3287 357.79907 0.52963257
335.44485 329.15656 6.2882996
touched pos: (325, 7)
touched pos: (386, 64)
touched pos: (482, 86)
send touch flag: 2 1 1 1 0 0 0 0 0 0 0 
touched_detection: 0.0040
computing time: 0.013962745666503906
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0050
send fingertips pos: 1 329 7 390 59 485 82 671 7 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
438.10153 437.8534 0.24813843
364.20477 360.2139 3.9908752
344.08405 343.17642 0.9076233
284.7511 284.70908 0.042022705
touched pos: (329, 7)
touched pos: (390, 59)
touched pos: (485, 82)
touched pos: (671, 7)
send touch flag: 2 1 1 1 1 0 0 0 0 0 0 
touched_detection: 0.0040
computing time: 0.012965202331542969
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0120
send fingertips pos: 1 667 374 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

skinImg: 0.0020
fingertips: 0.0110
send fingertips pos: 1 183 106 376 131 310 267 -1 -1 -1 -1 144 145 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
250.17679 244.46974 5.7070465
209.0887 208.0326 1.0561066
253.97891 245.54436 8.434555
touched pos: (183, 106)
touched pos: (310, 267)
touched pos: (144, 145)
send touch flag: 2 1 0 1 0 0 1 0 0 0 0 
touched_detection: 0.0020
computing time: 0.015957117080688477
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0090
send fingertips pos: 1 189 102 376 131 263 277 -1 -1 -1 -1 150 205 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
260.09814 253.93298 6.165161
207.002 206.21442 0.7875824
256.39117 255.57286 0.8183136
touched pos: (189, 102)
touched pos: (263, 277)
touched pos: (150, 205)
send touch flag: 2 1 0 1 0 0 1 0 0 0 0 
touched_detection: 0.0040
computing time: 0.01495981216430664
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0070
send fingertips pos: 1 151 203 373 133 234 264 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
trac

Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0130
send fingertips pos: 1 200 94 157 182 377 140 -1 -1 332 246 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
274.9885 272.50702 2.4814758
277.45145 275.7277 1.7237549
217.72253 216.52202 1.2005157
touched pos: (200, 94)
touched pos: (157, 182)
touched pos: (332, 246)
send touch flag: 2 1 1 0 0 1 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.01695394515991211
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0070
send fingertips pos: 1 158 183 -1 -1 366 131 -1 -1 233 248 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
277.58173 275.48578 2.0959473
218.63344 217.74515 0.8882904
235.42566 225.93277 9.492889
touched pos: (158, 183)
touched pos: (366, 131)
touched pos: (233, 248)
send touch flag: 2 1 0 1 0 1 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.012965202331542969
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0080
send fingertips pos: 1 157 183 -1 -1 371 217 -1 -1 231 246 -1 -1 

skinImg: 0.0020
fingertips: 0.0110
send fingertips pos: 1 198 84 144 161 -1 -1 156 106 325 240 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
260.7454 260.2634 0.48199463
256.22595 256.06787 0.15808105
253.68526 246.07025 7.6150055
214.11945 213.85506 0.26438904
touched pos: (198, 84)
touched pos: (144, 161)
touched pos: (156, 106)
touched pos: (325, 240)
send touch flag: 2 1 1 0 1 1 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.01595759391784668
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0060
send fingertips pos: 1 197 84 148 171 -1 -1 155 105 231 243 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
274.5857 273.1953 1.3903809
260.34262 256.7889 3.553711
250.58038 243.35065 7.2297363
230.53552 221.77722 8.758301
touched pos: (197, 84)
touched pos: (148, 171)
touched pos: (155, 105)
touched pos: (231, 243)
send touch flag: 2 1 1 0 1 1 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.011968374252319336
Start computing...
(376, 672)
skinImg: 0.0020
fingert

...

send fingertips pos: 1 56 111 314 69 342 4 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
168.07547 165.12987 2.9456024
touched pos: (56, 111)
touched pos: (342, 4)
send touch flag: 2 1 0 1 0 0 0 0 0 0 0 
touched_detection: 0.0030
computing time: 0.010970592498779297
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0025
send fingertips pos: 1 69 111 316 68 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.006489992141723633
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0040
send fingertips pos: 1 69 111 316 66 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0000
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.007978677749633789
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0020
send fingertips pos: 1 8 78 342 4 53 109 315 68 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
167.94238 

skinImg: 0.0020
fingertips: 0.0060
send fingertips pos: 1 6 77 28 96 48 108 312 71 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
152.09158 150.39726 1.6943207
155.7402 156.99274 -1.252533
touched pos: (6, 77)
touched pos: (48, 108)
send touch flag: 2 1 0 1 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.0109710693359375
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0040
send fingertips pos: 1 6 77 316 67 48 108 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0000
175.38246 171.42316 3.9593048
touched pos: (48, 108)
send touch flag: 2 0 0 1 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.007979154586791992
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0040
send fingertips pos: 1 2 75 79 110 316 68 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.007979154586791992
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0040
s

send fingertips pos: 1 6 79 27 97 56 111 315 71 344 8 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0020
168.20775 162.70305 5.5046997
touched pos: (344, 8)
send touch flag: 2 0 0 0 0 1 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.008976221084594727
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0020
send fingertips pos: 1 10 81 21 91 62 110 -1 -1 314 67 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.005983591079711914
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0030
send fingertips pos: 1 10 81 29 97 62 110 -1 -1 315 65 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
send touch flag: 2 0 0 0 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.006981372833251953
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0030
send fingertips pos: 1 9 81 55 112 315 71 -1 -1 343 10 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
166.3987 161.27747 5.121231
touch

fingertips: 0.0080
send fingertips pos: 1 85 113 348 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
169.27724 167.59276 1.6844788
touched pos: (348, 1)
send touch flag: 2 0 1 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.011967897415161133
Start computing...
(376, 672)
skinImg: 0.0020
fingertips: 0.0050
send fingertips pos: 1 38 102 337 17 316 64 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0010
162.31413 160.80434 1.5097961
touched pos: (337, 17)
send touch flag: 2 0 1 0 0 0 0 0 0 0 0 
touched_detection: 0.0020
computing time: 0.009973526000976562
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0030
send fingertips pos: 1 38 102 -1 -1 321 52 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 
tracking: 0.0000
153.06871 150.81049 2.2582245
touched pos: (321, 52)
send touch flag: 2 0 0 1 0 0 0 0 0 0 0 
touched_detection: 0.0010
computing time: 0.004986763000488281
Start computing...
(376, 672)
skinImg: 0.0010
fingertips: 0.0040
send finge

KeyboardInterrupt: 

In [ ]:
a = np.zeros((720,1280))
a[200:520,300:980] = 255
x = 50
a[x:x+35,x:x+35] = 255
x = 600
a[x:x+7,x:x+7] = 255
cv2.imshow("A",a.astype(np.uint8))
cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
# It need to restart than run ==> socket.sendall would not break

In [ ]:
t_time = time.time()
np.abs(a)>2
print('touched_detection: %.4f'%(time.time() - t_time))


t_time = time.time()
np.logical_or(a>2,a<2)
print('touched_detection: %.4f'%(time.time() - t_time))



x1 = np.logical_or(a,a<2)
x2 = np.logical_or(a,a>2)
x3 = np.logical_or(a,a<2)
x4 = np.logical_or(a,a>2)
x5 = np.logical_or(a,a<2)



t_time = time.time()
check_1_8 = np.array((x1, x2, x3, x4, x5))
Threshold_based_edge = np.logical_or.reduce(check_1_8)
print('touched_detection: %.4f'%(time.time() - t_time))

x1+ x2+ x3